## Vulnerability in the US at Census Tract Level

In [ ]:
import geopandas as gpd
import pandas as pd
import pathlib

import matplotlib.pyplot as plt

from cartopy import crs as ccrs

import os
import sys
import subprocess
import datetime
import platform
import datetime

In [ ]:
albo = ccrs.AlbersEqualArea(
    central_longitude=-96,
    central_latitude=37.5,
    false_easting=0.0,
    false_northing=0.0,
    standard_parallels=(29.5, 45.5),
    globe=None,
)

In [ ]:
import glob
path=r'Shapefile'

file=glob.glob(os.path.join(path, 'cb_2018_**_tract_500k.shp'))
# print(file)
print(len(file))

In [ ]:
## Import the census tracts

basemap= gpd.GeoDataFrame()
for f in file:
    basemap = pd.concat([basemap, gpd.read_file(f)], axis=0)
    
basemap = basemap[['GEOID','AFFGEOID','STATEFP','geometry']]
basemap.head()

In [ ]:
basemap.crs

In [ ]:
## Change the projection
basemap = basemap.to_crs(albo.proj4_init)

In [ ]:
## Change the position of Alaska and some islands

m = basemap.STATEFP =='15'
basemap[m] = basemap[m].set_geometry(basemap[m].translate(5000000,-1500000))

n = basemap.STATEFP == '02'
basemap[n] = basemap[n].set_geometry(basemap[n].scale(.5,.5,.5,origin=(-3000000, 4000000)).translate(1000000, -4800000))

State Boundary used in Maps

In [ ]:
## Import the state boundaries
state_boundary = gpd.read_file('cb_2018_us_state_500k/cb_2018_us_state_500k.shp')
state_boundary.plot()

In [ ]:
state_boundary = state_boundary.to_crs(albo.proj4_init)

m = state_boundary.STATEFP =='15'
state_boundary[m] = state_boundary[m].set_geometry(state_boundary[m].translate(5000000,-1500000))

n = state_boundary.STATEFP == '02'
state_boundary[n] = state_boundary[n].set_geometry(state_boundary[n].scale(.5,.5,.5,origin=(-3000000, 4000000)).translate(1000000, -4800000))

# Exposure

In [ ]:
## Import data

exposure = pd.read_csv('Exposure_0412.csv', dtype = {'State':str,"GEOID":str,"GEO_ID":str})
exposure.head()

In [ ]:
exposure_map = pd.merge(exposure,basemap,on='GEOID',how='right')
exposure_map = gpd.GeoDataFrame(exposure_map,crs = albo.proj4_init)
exposure_map.crs

In [ ]:
exposure_map.info()

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
# plt.rc('font',family='Times New Roman')

ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

exposure_map.plot(
    column='min-max',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=5,
    legend_kwds={'loc': 'upper left',
                 'title': 'Exposure',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # Title of legend
                 'fontsize': 11, # Font of legend
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Exposure', fontdict={'family':'Times New Roman','weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Exposure_USA.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

exposure_map.plot(
    column='fuel',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Fuel Consumption (gallons)',
                 "fmt": "{:.2f}",
                 'title_fontsize': 10, 
                 'fontsize': 10, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Fuel Consumption in the US', fontdict={'weight':'bold','size': 20}, pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Fuel Consumption_USA.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

exposure_map.plot(
    column='Price',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
    k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Fuel Price ($/gallon)',
                 "fmt": "{:.3f}",
                 'title_fontsize': 12, 
                 'fontsize': 12, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Fuel Price in the US', fontdict={'weight':'bold','size': 20}, pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Price_USA.png', facecolor='w', dpi=500)

# Sensitivity

In [ ]:
sensitivity = pd.read_csv("Sensitivity-0409.csv", dtype={"Geocode": str})

In [ ]:
sensitivity.head()

In [ ]:
sensitivity_map = pd.merge(sensitivity,basemap,left_on='Geocode',right_on='GEOID',how='right').drop(['Geocode'],axis=1)

In [ ]:
sensitivity_map = gpd.GeoDataFrame(sensitivity_map,crs = albo.proj4_init)

In [ ]:
sensitivity_map.crs

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

sensitivity_map.plot(
    column='min_max_sens',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Sensitivity',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Sensitivity', fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Sensitivity_USA.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

sensitivity_map.plot(
    column='X7',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'X7',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Sensitivity', fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/X7.png', facecolor='w', dpi=500)

# Adaptive Capacity

In [ ]:
adaptive_capacity = pd.read_csv('all_ac_data_0516.csv',dtype={"GEO_ID":str})
adaptive_capacity.head()

In [ ]:
adaptive_capacity_map = pd.merge(adaptive_capacity,basemap,left_on='GEO_ID',right_on='GEOID',how='right').drop(['GEO_ID'],axis=1)
adaptive_capacity_map.info()

In [ ]:
adaptive_capacity_map = gpd.GeoDataFrame(adaptive_capacity_map,crs = albo.proj4_init)
adaptive_capacity_map.crs

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

adaptive_capacity_map.plot(
    column='AC_score',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Adaptive Capacity',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Adaptive Capacity',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Adaptive Capacity_USA.png', facecolor='w', dpi=500)

# Vulnerabilty Score

In [ ]:
e = exposure_map[['GEOID','min-max']]
s = sensitivity_map[['GEOID','min_max_sens']]
ac = adaptive_capacity_map[['GEOID','AC_score']]

In [ ]:
vulnerablity = pd.merge(e, s, on="GEOID")
vulnerablity = pd.merge(vulnerablity, ac, on="GEOID")
vulnerablity.head()

In [ ]:
vulnerablity.info()

In [ ]:
vulnerablity.fillna(0, inplace=True)

In [ ]:
vulnerablity['vul_score_mulpi'] = vulnerablity.apply(lambda x: x["min-max"]*(x["min_max_sens"]-x["AC_score"]), axis=1)

In [ ]:
vulnerablity['vul_score_add'] = vulnerablity.apply(lambda x: x["min-max"]+x["min_max_sens"]-x["AC_score"], axis=1)

In [ ]:
vulnerablity.head()

In [ ]:
vulnerablity.to_csv('Vulnerability Score.csv')

In [ ]:
vulnerablity_map = pd.merge(vulnerablity,basemap,on='GEOID')
vulnerablity_map = gpd.GeoDataFrame(vulnerablity_map,crs = albo.proj4_init)

In [ ]:
vulnerablity_map.crs

In [ ]:
vulnerablity_map_pos = vulnerablity_map[vulnerablity_map['vul_score_mulpi']>=0]
vulnerablity_map_neg = vulnerablity_map[vulnerablity_map['vul_score_mulpi']<0]
print(vulnerablity_map_pos['vul_score_mulpi'].mean(),vulnerablity_map_pos['vul_score_mulpi'].std())
print(vulnerablity_map_neg['vul_score_mulpi'].mean(),vulnerablity_map_neg['vul_score_mulpi'].std())

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

vulnerablity_map.plot(
    column='vul_score_add',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, 
                 'fontsize': 11, 
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Vul_score_add.png', facecolor='w', dpi=500)

# Weight Testing

In [ ]:
vul_weidghted = pd.merge(e, s, on="GEOID")
vul_weidghted = pd.merge(vul_weidghted, ac, on="GEOID")
vul_weidghted.head()

In [ ]:
vul_weidghted.fillna(0, inplace=True)

In [ ]:
# min-max: exposure, min_max_sens: sensitivity; AC_score: adaptive capacity
# Suppose the weight are 0.6 , 0.2, 0.2
vul_weidghted['vul_mulpi_622'] = vul_weidghted.apply(lambda x: 1.8*x["min-max"]*(0.6*x["min_max_sens"]-0.6*x["AC_score"]), axis=1)
vul_weidghted['vul_add_622'] = vul_weidghted.apply(lambda x: 1.8*x["min-max"]+0.6*x["min_max_sens"]-0.6*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_622'].corr(vulnerablity['vul_score_mulpi'])

In [ ]:
vul_weidghted['vul_add_622'].corr(vulnerablity['vul_score_add'])

In [ ]:
# Suppose the weight are 0.2 , 0.6, 0.2
vul_weidghted['vul_mulpi_262'] = vul_weidghted.apply(lambda x: 0.6*x["min-max"]*(1.8*x["min_max_sens"]-0.6*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_add_262'] = vul_weidghted.apply(lambda x: 0.2*x["min-max"]+0.6*x["min_max_sens"]-0.2*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_262'].corr(vulnerablity['vul_score_mulpi'])

In [ ]:
vul_weidghted['vul_add_262'].corr(vulnerablity['vul_score_add'])

In [ ]:
# Suppose the weight are 0.2 , 0.2, 0.6
vul_weidghted['vul_mulpi_226'] = vul_weidghted.apply(lambda x: 0.2*x["min-max"]*(0.2*x["min_max_sens"]-0.6*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_add_226'] = vul_weidghted.apply(lambda x: 0.2*x["min-max"]+0.2*x["min_max_sens"]-0.6*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_226'].corr(vulnerablity['vul_score_mulpi'])

In [ ]:
vul_weidghted['vul_add_226'].corr(vulnerablity['vul_score_add'])

In [ ]:
vul_weidghted.to_excel('vul_weidght_test.xlsx')

# EV Adoption in IL

In [ ]:
cv_ev_ex = pd.read_excel('IL CV-EV/IL_CV2EV_base.xlsx',dtype={'CountyFIPs':str})
cv_ev_ex.head()

In [ ]:
ex_IL = cv_ev_ex[['CountyFIPs','County', '2020_CV Costexpo_nor', '2030_CV Costexpo_nor',
               '2040_CV Costexpo_nor','2050_CV Costexpo_nor','2020_EV Costexpo_nor','2030_EV Costexpo_nor',
               '2040_EV Costexpo_nor','2050_EV Costexpo_nor']]
ex_IL.head()

In [ ]:
s_IL = sensitivity_map[['GEOID','min_max_sens']]
ac_IL = adaptive_capacity_map[['GEOID','AC_score']]

In [ ]:
pop = pd.read_csv('IL CV-EV/latch_2017.csv',dtype={'geocode':str})
for i in range(len(pop)):
    pop.loc[i,'geocode'] = pop.loc[i,'geocode'].zfill(11)
pop.head()

In [ ]:
col_name=pop.columns.tolist()
col_name.insert(1,'StateFIPs')
col_name.insert(2,'CountyFIPs')
col_name.insert(3,'CountyPOP')
pop=pop.reindex(columns=col_name)
for i in range(len(pop)):
    pop.loc[i,'StateFIPs'] = str(pop.loc[i,'geocode'])[:2]
    pop.loc[i,'CountyFIPs'] = str(pop.loc[i,'geocode'])[:5]

In [ ]:
pop_IL = pop[(pop.StateFIPs == "17")]
pop_IL = pop_IL.reset_index()

In [ ]:
pop_county = pop_IL.groupby(['CountyFIPs'])[['tot_pop']].sum()
pop_county = pop_county.reset_index()
pop_county.head()

In [ ]:
for i in range(len(pop_IL)):
    for j in range(len(pop_county)):
        if pop_IL.loc[i,'CountyFIPs'] == pop_county.loc[j,'CountyFIPs']:
            pop_IL.loc[i,'CountyPOP'] = pop_county.loc[j,'tot_pop']

In [ ]:
pop_IL

In [ ]:
s_IL = pd.merge(s_IL,pop_IL,left_on='GEOID',right_on='geocode',how="left")
ac_IL = pd.merge(ac_IL,pop_IL,left_on='GEOID',right_on='geocode',how="left")

In [ ]:
s_IL = s_IL[(s_IL.StateFIPs == "17")]
ac_IL = ac_IL[(ac_IL.StateFIPs == "17")]

In [ ]:
s_IL['sens-new'] = s_IL.apply(lambda x: x['min_max_sens'] * (x['tot_pop']/x['CountyPOP']),axis=1)
ac_IL['ac-new'] = ac_IL.apply(lambda x: x['AC_score'] * (x['tot_pop']/x['CountyPOP']),axis=1)

In [ ]:
s_county = s_IL.groupby(['CountyFIPs'])[['sens-new']].sum()
s_county.head()

In [ ]:
ac_county = ac_IL.groupby(['CountyFIPs'])[['ac-new']].sum()
ac_county.head()

In [ ]:
s_county = s_county.reset_index()
ac_county = ac_county.reset_index()

In [ ]:
minsens = s_county['sens-new'].min()
maxsens = s_county['sens-new'].max()
minac = ac_county['ac-new'].min()
maxac = ac_county['ac-new'].max()
s_county['sens-new'] = s_county.apply(lambda x: (x['sens-new'] - minsens)/(maxsens-minsens),axis=1)
ac_county['ac-new'] = ac_county.apply(lambda x: (x['ac-new'] - minac)/(maxac-minac),axis=1)

In [ ]:
ac_county.head()

In [ ]:
vul_score = pd.merge(ex_IL,s_county,on='CountyFIPs')
vul_score = pd.merge(vul_score,ac_county,on='CountyFIPs')
vul_score.head()

In [ ]:
vul_score.describe()

In [ ]:
vul_score.fillna(0, inplace=True)

In [ ]:
vul_score['2020CV'] = vul_score.apply(lambda x: x["2020_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2030CV'] = vul_score.apply(lambda x: x["2030_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2040CV'] = vul_score.apply(lambda x: x["2040_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2050CV'] = vul_score.apply(lambda x: x["2050_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2020EV'] = vul_score.apply(lambda x: x["2020_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2030EV'] = vul_score.apply(lambda x: x["2030_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2040EV'] = vul_score.apply(lambda x: x["2040_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2050EV'] = vul_score.apply(lambda x: x["2050_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)

In [ ]:
vul_score.head()

In [ ]:
vul_score.to_excel('IL CV-EV/vul_score CV-EV_base.xlsx')